In [10]:
import numpy as np
# from scipy.interpolate import griddata
# from scipy.spatial import cKDTree
import laspy
from scipy import ndimage
from tqdm import tqdm
from matplotlib import pyplot as plt
import os

In [2]:
folder_path = "/home/chazelam/Code/LiDAR/5+1_split_output"
file_names = os.listdir(folder_path)
# file_name = os.path.join(folder_path, file_names[0])
file_name = os.path.join(folder_path, "tile_4_4.las")


las = laspy.read(file_name)
points = np.vstack((las.x, las.y, las.z)).T
print(f"{len(points):,}")

43,005,548


In [66]:
cell_size=0.5
max_window_size=15
initial_threshold=0.3
slope=0.2

In [ ]:
print(f"x: {las.header.x_min} - {las.header.x_max}")
print(f"y: {las.header.y_min} - {las.header.y_max}")

x: 0.0 - 10.849
y: -3.029 - 3.97


In [18]:
min_x, min_y = np.min(points[:, :2], axis=0)
max_x, max_y = np.max(points[:, :2], axis=0)

print(f"x: {min_x} - {max_x}")
print(f"y: {min_y} - {max_y}")

x: 0.0 - 10.849
y: -3.029 - 3.97


In [34]:
cols = np.ceil((max_x - min_x) / cell_size).astype(int)[0]
rows = np.ceil((max_y - min_y) / cell_size).astype(int)[0]

print(cols)
print(rows)

22
14


In [ ]:
grid_old = np.full((rows, cols), np.inf)
for x, y, z in points:
    col = int((x - min_x) / cell_size)
    row = int((y - min_y) / cell_size)
    if 0 <= row < rows and 0 <= col < cols:
        if z < grid_old[row, col]:
            grid_old[row, col] = z

/tmp/ipykernel_8049/1975137510.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  col = int((x - min_x) / cell_size)
/tmp/ipykernel_8049/1975137510.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  row = int((y - min_y) / cell_size)


In [ ]:
grid = np.full((rows, cols), np.inf, dtype=np.float32)

# индексы всех точек (векторно)
col_idx = np.floor((points[:,0] - min_x) / cell_size).astype(np.int64)
row_idx = np.floor((points[:,1] - min_y) / cell_size).astype(np.int64)

# маска точек внутри области
valid = (row_idx >= 0) & (row_idx < rows) & (col_idx >= 0) & (col_idx < cols)
row_idx = row_idx[valid]
col_idx = col_idx[valid]
z_vals  = points[valid, 2].astype(np.float32)

# сводим 2D индексы в 1D для применения minimum.at
flat_idx = row_idx * cols + col_idx
flat_grid = grid.ravel()

# применяем минимум по индексам (в C, быстро)
np.minimum.at(flat_grid, flat_idx, z_vals)

# возвращаем 2D
grid = flat_grid.reshape((rows, cols))

In [ ]:
dif = np.subtract(grid, grid_old)
dif = dif[~np.isnan(dif)]
print(dif.max())

1.754760745598105e-07


/tmp/ipykernel_8049/2077071434.py:1: RuntimeWarning: invalid value encountered in subtract
  dif = np.subtract(grid, grid_2)


In [73]:
def update_ground_mask(points, 
                       grid_opened, 
                       ground_mask, 
                       min_x, min_y, 
                       cell_size, 
                       current_threshold):
    
    rows, cols = grid_opened.shape
    x, y, z = points[:, 0], points[:, 1], points[:, 2]

    # Преобразуем в индексы ячеек (векторно)
    col_idx = np.floor((x - min_x) / cell_size).astype(np.int32)
    row_idx = np.floor((y - min_y) / cell_size).astype(np.int32)

    # Маска точек, попадающих внутрь сетки
    valid = (row_idx >= 0) & (row_idx < rows) & (col_idx >= 0) & (col_idx < cols)
    row_idx, col_idx, z_valid = row_idx[valid], col_idx[valid], z[valid]

    # Извлекаем значения из открытой поверхности
    opened_vals = grid_opened[row_idx, col_idx]

    # Проверяем условие
    mask_valid = (z_valid - opened_vals) <= current_threshold

    # Обновляем исходную маску
    ground_mask[valid] |= mask_valid

In [ ]:
# Дольше 15 минут
# Проблема в переборе точек

ground_mask = np.zeros(len(points), dtype=bool)

window_sizes = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
    
for window_size in window_sizes:
    if window_size > max_window_size:
        break
        
    footprint = np.ones((window_size, window_size))
    eroded = ndimage.grey_erosion(grid, footprint=footprint)
    opened = ndimage.grey_dilation(eroded, footprint=footprint)
    
    current_threshold = initial_threshold + slope * (window_size / 2) * cell_size
    
    for i, (x, y, z) in enumerate(points):
        col = int((x - min_x) / cell_size)
        row = int((y - min_y) / cell_size)
        
        if 0 <= row < rows and 0 <= col < cols:
            if z - opened[row, col] <= current_threshold:
                ground_mask[i] = True

In [ ]:
ground_mask = np.zeros(len(points), dtype=bool)

window_sizes = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
    
for window_size in window_sizes:
    if window_size > max_window_size:
        break
        
    footprint = np.ones((window_size, window_size))
    eroded = ndimage.grey_erosion(grid, footprint=footprint)
    opened = ndimage.grey_dilation(eroded, footprint=footprint)
    current_threshold = initial_threshold + slope * (window_size / 2) * cell_size

    update_ground_mask(points, 
                       opened, 
                       ground_mask, 
                       min_x, min_y,
                       cell_size,
                       current_threshold)
    
# Invert ground
non_ground_mask = ~ground_mask

print(f"Non ground points: {np.sum(non_ground_mask):,}")
print(f"Ground points: {np.sum(ground_mask):,}")

In [75]:
def save_las_points(
    original_las: laspy.LasData,
    points: np.ndarray,
    output_path: str,
    copy_attributes: bool = True):
    
    # создаём новый заголовок на основе оригинала
    header = laspy.LasHeader(
        point_format=original_las.header.point_format,
        version=original_las.header.version
    )
    header.scales = original_las.header.scales
    header.offsets = original_las.header.offsets

    # создаём новый LAS-объект
    new_las = laspy.LasData(header)

    # записываем координаты
    new_las.x = points[:, 0]
    new_las.y = points[:, 1]
    new_las.z = points[:, 2]

    # при необходимости копируем остальные поля
    if copy_attributes:
        attrs = [
            'intensity', 'return_number', 'number_of_returns',
            'classification', 'user_data', 'gps_time',
            'red', 'green', 'blue'
        ]
        for attr in attrs:
            if hasattr(original_las, attr):
                data = getattr(original_las, attr)
                if len(data) == len(original_las.points):
                    new_las[attr] = data[:len(points)]  # если точек меньше, копируем соответствующее количество

    # сохраняем
    new_las.write(output_path)
    print(f"Saved {len(points):,} points to {output_path}")


In [ ]:
non_ground_points = points[non_ground_mask]

save_las_points(las, points[ground_mask], "ground.las")
save_las_points(las, non_ground_points, "non_ground.las")

Non ground points: 18878285
Ground points: 24127263
Saved 24,127,263 points to ground.las
Saved 18,878,285 points to non_ground.las
